# <center> Introduction to Spark In-memory Computing via Python PySpark </center>

In [1]:
!bash launch_spark_cluster.sh

Launching Spark cluster with the following parameters:
Master Node: node0314.palmetto.clemson.edu
Slave Nodes:
node0317.palmetto.clemson.edu
node0355.palmetto.clemson.edu
node0442.palmetto.clemson.edu
Temporary dir: /local_scratch/pbs.8742780.pbs02
Memory per worker (GB): 13G
Cores per worker: 8
Num workers: 3
. /home/aamle/software/spark-2.4.5-bin-hadoop2.7/sbin/start-all.sh -h node0314.palmetto.clemson.edu -d /local_scratch/pbs.8742780.pbs02 -m 13G -c 3
starting org.apache.spark.deploy.master.Master, logging to /home/aamle/software/spark-2.4.5-bin-hadoop2.7/logs/spark-aamle-org.apache.spark.deploy.master.Master-1-node0314.palmetto.clemson.edu.out
node0317.palmetto.clemson.edu: starting org.apache.spark.deploy.worker.Worker, logging to /home/aamle/software/spark-2.4.5-bin-hadoop2.7/logs/spark-aamle-org.apache.spark.deploy.worker.Worker-1-node0317.palmetto.clemson.edu.out
node0355.palmetto.clemson.edu: starting org.apache.spark.deploy.worker.Worker, logging to /home/aamle/software/spar

In [2]:
import sys
import os
import pyspark

env_spark_home=os.path.join(os.environ['HOME'],"software","spark-2.4.5-bin-hadoop2.7")
env_spark_conf_dir=os.path.join(env_spark_home,"conf")
env_pyspark_python=os.path.join("/software","anaconda3","5.1.0","bin","python")

os.environ['SPARK_HOME'] = env_spark_home
os.environ['SPARK_CONF_DIR'] = env_spark_conf_dir
os.environ['PYSPARK_PYTHON'] = env_pyspark_python

fp = open(os.path.join(env_spark_conf_dir,"master"))
node_list = fp.readlines()

import pyspark
conf = pyspark.SparkConf()
conf.setMaster("spark://" + node_list[0].strip() + ":7077")
conf.setAppName('big-data-workshop')
conf.set("spark.driver.memory","5g")
conf.set("spark.executor.instances", "3")
conf.set("spark.executor.memory","13g")
conf.set("spark.executor.cores","8")

sc = pyspark.SparkContext(conf=conf)

print(sc)

<SparkContext master=spark://node0314.palmetto.clemson.edu:7077 appName=big-data-workshop>


### Airlines Data

**Spark SQL**
- Spark module for structured data processing
- provides more information about the structure of both the data and the computation being performed for additional optimization
- execute SQL queries written using either a basic SQL syntax or HiveQL

**DataFrame**
- a distributed collection of data organized into named columns
- conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood
- can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs

In [3]:
sqlContext = pyspark.SQLContext(sc)
sqlContext

In [4]:
airlines = sqlContext.read.format("com.databricks.spark.csv")\
    .option("header", "true")\
    .option("inferschema", "true")\
    .load("/zfs/citi/airlines/data/")\
    .cache()

In [5]:
%%time
airlines.count()

CPU times: user 8.33 ms, sys: 6.86 ms, total: 15.2 ms
Wall time: 2min 16s


123534969

In [6]:
%%time
airlines.count()

CPU times: user 217 µs, sys: 2.76 ms, total: 2.98 ms
Wall time: 285 ms


123534969

In [7]:
airlines.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Carr

You can interact with a DataFrame via SQLContext using SQL statements by registering the DataFrame as a table

In [8]:
airlines.registerTempTable("airlines")

*How many unique airlines are there?*

In [9]:
uniqueAirline = sqlContext.sql("SELECT DISTINCT UniqueCarrier \
                                FROM airlines")
uniqueAirline.show()

+-------------+
|UniqueCarrier|
+-------------+
|           UA|
|           EA|
|           PI|
|           PS|
|           AA|
|           NW|
|           EV|
|           B6|
|           HP|
|           TW|
|           DL|
|           OO|
|           F9|
|           YV|
|           TZ|
|           US|
|           AQ|
|           MQ|
|           OH|
|           HA|
+-------------+
only showing top 20 rows



*Calculate how many flights completed by each carrier over time*

In [10]:
%%time
carrierFlightCount = sqlContext.sql("SELECT UniqueCarrier, COUNT(UniqueCarrier) AS FlightCount \
                                    FROM airlines GROUP BY UniqueCarrier")
carrierFlightCount.show()

+-------------+-----------+
|UniqueCarrier|FlightCount|
+-------------+-----------+
|           UA|   13299817|
|           EA|     919785|
|           PI|     873957|
|           PS|      83617|
|           AA|   14984647|
|           NW|   10292627|
|           EV|    1697172|
|           B6|     811341|
|           HP|    3636682|
|           TW|    3757747|
|           DL|   16547870|
|           OO|    3090853|
|           F9|     336958|
|           YV|     854056|
|           TZ|     208420|
|           AQ|     154381|
|           US|   14075530|
|           MQ|    3954895|
|           OH|    1464176|
|           HA|     274265|
+-------------+-----------+
only showing top 20 rows

CPU times: user 3.32 ms, sys: 670 µs, total: 3.99 ms
Wall time: 3.62 s


*How do you display full carrier names?*

In [11]:
carriers = sqlContext.read.format("com.databricks.spark.csv")\
    .option("header", "true")\
    .option("inferschema", "true")\
    .load("/zfs/citi/airlines/metadata/carriers.csv")\
    .cache()
carriers.registerTempTable("carriers")

In [12]:
carriers.printSchema()

root
 |-- Code: string (nullable = true)
 |-- Description: string (nullable = true)



In [13]:
%%time
carrierFlightCountFullName = sqlContext.sql("SELECT c.Description, a.UniqueCarrier, COUNT(a.UniqueCarrier) AS FlightCount \
                                    FROM airlines AS a \
                                    INNER JOIN carriers AS c \
                                    ON c.Code = a.UniqueCarrier \
                                    GROUP BY a.UniqueCarrier, c.Description \
                                    ORDER BY a.UniqueCarrier")
carrierFlightCountFullName.show()

+--------------------+-------------+-----------+
|         Description|UniqueCarrier|FlightCount|
+--------------------+-------------+-----------+
|Pinnacle Airlines...|           9E|     521059|
|American Airlines...|           AA|   14984647|
| Aloha Airlines Inc.|           AQ|     154381|
|Alaska Airlines Inc.|           AS|    2878021|
|     JetBlue Airways|           B6|     811341|
|Continental Air L...|           CO|    8145788|
|    Independence Air|           DH|     693047|
|Delta Air Lines Inc.|           DL|   16547870|
|Eastern Air Lines...|           EA|     919785|
|Atlantic Southeas...|           EV|    1697172|
|Frontier Airlines...|           F9|     336958|
|AirTran Airways C...|           FL|    1265138|
|Hawaiian Airlines...|           HA|     274265|
|America West Airl...|           HP|    3636682|
|Midway Airlines I...|       ML (1)|      70622|
|American Eagle Ai...|           MQ|    3954895|
|Northwest Airline...|           NW|   10292627|
|         Comair Inc

*What is the averaged departure delay time for each airline?*

In [14]:
%%time
avgDepartureDelay = sqlContext.sql("SELECT FIRST(c.Description), FIRST(a.UniqueCarrier), AVG(a.DepDelay) AS AvgDepDelay \
                                    FROM airlines AS a \
                                    INNER JOIN carriers AS c \
                                    ON c.Code = a.UniqueCarrier \
                                    GROUP BY a.UniqueCarrier \
                                    ORDER BY a.UniqueCarrier")
avgDepartureDelay.show()

+-------------------------+---------------------------+-------------------+
|first(Description, false)|first(UniqueCarrier, false)|        AvgDepDelay|
+-------------------------+---------------------------+-------------------+
|     Pinnacle Airlines...|                         9E| 7.9279144892173035|
|     American Airlines...|                         AA|  7.862321254420546|
|      Aloha Airlines Inc.|                         AQ| 1.5993176899118409|
|     Alaska Airlines Inc.|                         AS|  8.297235193754096|
|          JetBlue Airways|                         B6| 11.262714178314551|
|     Continental Air L...|                         CO|  7.695967155526857|
|         Independence Air|                         DH|  9.612639389688926|
|     Delta Air Lines Inc.|                         DL|  7.593716274369933|
|     Eastern Air Lines...|                         EA|  8.674050565435543|
|     Atlantic Southeas...|                         EV| 13.483736343326541|
|     Fronti

In [15]:
airlines.unpersist()

DataFrame[Year: int, Month: int, DayofMonth: int, DayOfWeek: int, DepTime: string, CRSDepTime: int, ArrTime: string, CRSArrTime: int, UniqueCarrier: string, FlightNum: int, TailNum: string, ActualElapsedTime: string, CRSElapsedTime: string, AirTime: string, ArrDelay: string, DepDelay: string, Origin: string, Dest: string, Distance: string, TaxiIn: string, TaxiOut: string, Cancelled: int, CancellationCode: string, Diverted: int, CarrierDelay: string, WeatherDelay: string, NASDelay: string, SecurityDelay: string, LateAircraftDelay: string]

In [16]:
sc.stop()

In [17]:
!bash stop_spark_cluster.sh

Stopping Spark cluster:
node0355.palmetto.clemson.edu: stopping org.apache.spark.deploy.worker.Worker
node0317.palmetto.clemson.edu: stopping org.apache.spark.deploy.worker.Worker
node0442.palmetto.clemson.edu: stopping org.apache.spark.deploy.worker.Worker
stopping org.apache.spark.deploy.master.Master
